In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from tempfile import TemporaryDirectory

import matplotlib as mpl
import seaborn as sns
import torch
import utils
import wandb
from matplotlib import pyplot as plt

import opf
import opf.dataset
from opf.hetero import HeteroGCN
from opf.modules import OPFLogBarrier

mpl.rcParams["text.usetex"] = False

sns.set_style("white")
sns.set_context("paper")

torch.set_float32_matmul_precision("medium")

In [4]:
run_uri = "wandb://katyamaz/opf/8uzjmnce"

In [5]:
base, path = run_uri.split("://")
if base != "wandb":
    raise ValueError("Only wandb runs are supported")

api = wandb.Api()
run = api.run(path)
run.config.update(data_dir="../data")

# initialize data
dm = opf.dataset.CaseDataModule(**run.config)
dm.setup("test")
assert dm.test_dataset is not None

# initialize model and initialize lazy layers
barrier = OPFLogBarrier(
    **run.config, model=HeteroGCN(dm.metadata(), -1, 4, **run.config)
)
barrier(dm.test_dataset[0])

# load checkpoint
with TemporaryDirectory() as tmpdir:
    artifact = api.artifact(f"katyamaz/opf/model-{run.id}:best")
    checkpoint_path = artifact.download(root=tmpdir)
    checkpoint = torch.load(Path(checkpoint_path) / "model.ckpt", map_location="cpu")
    barrier.load_state_dict(checkpoint["state_dict"], strict=True)

/home/kmazu/.cache/pypoetry/virtualenvs/opf-D4LT1ipO-py3.11/lib/python3.11/site-packages/torch_sparse/tensor.py:574: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(rowptr, col, value, self.sizes())
wandb:   1 of 1 files downloaded.  


In [6]:
# save_dir = f"./figures/{run.config['case_name']}/"
# os.makedirs(save_dir, exist_ok=True)


def save(title: str, fig=plt):
    pass
    # fig.savefig(save_dir + f"{title}.png")
    # fig.savefig(save_dir + f"{title}.pdf")

In [9]:
# Run test
barrier.detailed_metrics = True
outputs = utils.test(barrier, dm, accelerator="gpu")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/kmazu/.cache/pypoetry/virtualenvs/opf-D4LT1ipO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

ERROR: LoadError: The size of the variables is not the same.
Stacktrace:
 [1] error(s::String)
   @ Base ./error.jl:35
 [2] main()
   @ Main ~/opf/src/opf/project.jl:39
 [3] top-level scope
   @ ~/opf/src/opf/project.jl:55
in expression starting at /home/kmazu/opf/src/opf/project.jl:55


RuntimeError: shape '[32, 30]' is invalid for input of size 36

# Summary

In [7]:
df = outputs.copy()
display(
    df[
        [
            "test/equality/error_max",
            "test/inequality/error_max",
            "test/inequality/active_power/error_max",
            "test/inequality/reactive_power/error_max",
            "test/inequality/voltage_magnitude/error_max",
            "test/inequality/forward_rate/error_max",
            "test/inequality/backward_rate/error_max",
            "test/inequality/voltage_angle_difference/error_max",
        ]
    ].max()
)

print(
    f"""
GNN Cost: {df["test/cost"].mean():0.4f}

Maximum violation rate: {df["test/inequality/rate"].max():0.4f}
Rate of any violation: {(df["test/inequality/error_max"] > 1e-4).sum() / len(df):0.4f}
"""
)
# not implemented right now
# IPOPT Cost: {df["acopf/cost"].mean():0.4f}
# GNN/IPOPT: {(df["test/cost"]/df["acopf/cost"]).mean():0.4f}

NameError: name 'outputs' is not defined

# Histograms

In [8]:
from utils import FlowLayout

aspect = 1.618
ylabel = "Count / # of samples"
kwargs = dict(
    bins=10,
    stat="proportion",
    aspect=aspect,
    height=3.5 / aspect,
)


sns.displot(df, x="test/inequality/error_max", **kwargs)
plt.xlabel("Max inequality error")
plt.ylabel(ylabel)
save("error_max")

sns.displot(df, x="test/inequality/error_mean", **kwargs)
plt.xlabel("Mean inequality error")
plt.ylabel(ylabel)
save("error_mean")

# Cost improvement
# df["test/cost/improvement"] = df["test/cost"] / df["acopf/cost"]
# df["violation"] = df["test/inequality/rate"] > 1e-8
# sns.displot(df[~df["violation"]], x="test/cost/improvement", **kwargs)
# plt.xlabel("GNN / IPOPT cost ratio")
# plt.ylabel(ylabel)
# save("costs")

# map variable names to series names
fmt = "test/inequality/%s/error_max"
hist_dict = {
    "equality": ["test/equality/bus_power/error_max"],
    "gen": [fmt % "active_power", fmt % "reactive_power"],
    "vm": [fmt % "voltage_magnitude"],
    "rate": [fmt % "forward_rate", fmt % "backward_rate"],
    "vad": [fmt % "voltage_angle_difference"],
}

sns.displot(df["test/equality/bus_power/error_max"], **kwargs)
plt.xlabel("Bus power equality error")
plt.ylabel(ylabel)
save("error_equality")

power_df = df.melt(value_vars=[fmt % "active_power", fmt % "reactive_power"])
sns.displot(power_df, x="value", **kwargs)
plt.xlabel("Generated power error")
plt.ylabel(ylabel)
save("error_gen")

sns.displot(df[fmt % "voltage_magnitude"], **kwargs)
plt.xlabel("Voltage magnitude error")
plt.ylabel(ylabel)
save("error_vm")

flow_df = df.melt(value_vars=[fmt % "forward_rate", fmt % "backward_rate"])
sns.displot(flow_df, x="value", **kwargs)
plt.xlabel("Power rate limit error")
plt.ylabel(ylabel)
save("error_rate")

sns.displot(df[fmt % "voltage_angle_difference"], **kwargs)
plt.xlabel("Voltage angle difference error")
plt.ylabel(ylabel)
save("error_vad")

FlowLayout().all_open()

NameError: name 'df' is not defined

# Visualizing Violations

In [10]:
sort_term = "test/inequality/error_max"
quantile = 0.5

s = df[sort_term]
index = int((s.sort_values()[::-1] <= s.quantile(quantile)).idxmax())
print(sort_term, s[index])
print("Idx", index)
df.iloc[index][
    [
        "test/cost",
        "test/equality/bus_power/error_max",
        "test/equality/bus_reference/error_max",
        "test/inequality/error_max",
        "test/inequality/active_power/error_max",
        "test/inequality/reactive_power/error_max",
        "test/inequality/voltage_magnitude/error_max",
        "test/inequality/forward_rate/error_max",
        "test/inequality/backward_rate/error_max",
        "test/inequality/voltage_angle_difference/error_max",
    ]
]

NameError: name 'df' is not defined

In [ ]:
# perform inference on the test set
from opf.plot import plot_constraints
import opf.powerflow as pf

with torch.no_grad():
    dm.setup("test")
    dataset = dm.test_dataset
    assert isinstance(dataset, opf.dataset.StaticGraphDataset)
    data = dataset[index]
    V, S, Sd = barrier.float()(data)
    variables, _, _, _ = barrier._step_helper(
        V, S, Sd, data.powerflow_parameters, substitute_equality=True
    )
    constraints = pf.build_constraints(variables, data.powerflow_parameters)
    plots = plot_constraints(constraints)

AssertionError: 

In [ ]:
# import numpy as np

# dataset = dm.test_dataset
# assert isinstance(dataset, opf.dataset.StaticGraphDataset)

# n_bus = dataset[0].powerflow_parameters.n_bus
# casefile = dm.case_path.as_posix()

# V = np.zeros((len(dataset), n_bus, 2))
# Sg = np.zeros((len(dataset), n_bus, 2))
# Sd = np.zeros((len(dataset), n_bus, 2))
# with torch.no_grad():
#     for i in range(len(dataset)):
#         data = dataset[i]
#         _V, _Sg, _Sd = barrier.float()(data)
#         V[i, :] = torch.view_as_real(_V).squeeze().numpy()
#         Sg[i, :] = torch.view_as_real(_Sg).squeeze().numpy()
#         Sd[i, :] = torch.view_as_real(_Sd).squeeze().numpy()

# # project using Julia
# with TemporaryDirectory() as tempdir:
#     busfile = Path(tempdir) / "busfile.npz"
#     np.savez(busfile, V=V, Sg=Sg, Sd=Sd)
#     !julia ../scripts/project.jl --casefile {casefile} --busfile {busfile.as_posix()}
#     bus = np.load(busfile)
# V, Sg, Sd = bus["V"], bus["Sg"], bus["Sd"]

In [ ]:
# df["acopf/cost/relaxed"] = None
# for i in tqdm.tqdm(range(len(df))):
#     forward_error = df.iloc[i]["test/inequality/forward_rate/error_max"]
#     backward_error = df.iloc[i]["test/inequality/backward_rate/error_max"]

#     increase = 1 + torch.clamp((torch.maximum(variables.Sf.abs(), variables.St.abs()) - barrier.powerflow_parameters.rate_a) / barrier.powerflow_parameters.rate_a, min=0) \
#         .squeeze().detach().numpy()
#     net = barrier.net_wrapper.net
#     original = net.line.copy()
#     barrier.net_wrapper.set_load_sparse(variables.Sd.real.squeeze(), variables.Sd.imag.squeeze())

#     net.line["max_i_ka"] *= increase[:len(net.line)]
#     bus = torch.as_tensor(barrier.net_wrapper.optimal_ac(False)[0]).double().unsqueeze(0)
#     _, constraints, cost, _ = barrier._step_helper(
#                     *barrier.parse_bus(bus),
#                     variables.Sd,
#                     project_pandapower=False,
#     )
#     net.line = original
#     df["acopf/cost/relaxed"].iloc[i] = cost.item()
# print("IPOPT Original Cost:", df["acopf/cost"].mean())
# print("IPOPT Relaxed Cost:", df["acopf/cost/relaxed"].mean())
# print("GNN Cost", df["test/cost"].mean())